In [5]:
import cv2
import time
import csv
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort


In [10]:

# Load YOLOv11 model (replace with your trained model)
model = YOLO("my_model.pt")

# Initialize Deep SORT
tracker = DeepSort(max_age=30)

# Load video
cap = cv2.VideoCapture('clean.mp4')  # Replace with your video
fps = cap.get(cv2.CAP_PROP_FPS)
frame_index = 0

# Data store for logging
tracking_data = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_index += 1
    timestamp = frame_index / fps

    # Resize for YOLO input
    resized_frame = cv2.resize(frame, (640, 640))
    results = model(resized_frame)[0]

    detections = []
    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls_id))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=resized_frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        center_x = (l + r) // 2

        # Log first appearance
        if track_id not in tracking_data:
            tracking_data[track_id] = {
                "entry_time": timestamp,
                "positions": [center_x]
            }
        else:
            tracking_data[track_id]["positions"].append(center_x)

        # Draw rectangle and label
        cv2.rectangle(resized_frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(resized_frame, f'Bird ID {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Bird Tracking", resized_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Save CSV log
with open('bird_tracking_log.csv', 'w', newline='') as csvfile:
    fieldnames = ['Bird ID', 'Entry Time (s)', 'Movement']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for track_id, data in tracking_data.items():
        entry_time = round(data['entry_time'], 2)
        positions = data['positions']
        movement = "Stayed"

        if positions[-1] - positions[0] > 30:
            movement = "Left to Right"
        elif positions[0] - positions[-1] > 30:
            movement = "Right to Left"

        writer.writerow({
            'Bird ID': track_id,
            'Entry Time (s)': entry_time,
            'Movement': movement
        })

print("Tracking log saved to bird_tracking_log.csv ✅")



0: 640x640 8 Birds, 1756.0ms
Speed: 141.9ms preprocess, 1756.0ms inference, 36.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 Birds, 829.0ms
Speed: 16.5ms preprocess, 829.0ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 Birds, 858.6ms
Speed: 27.6ms preprocess, 858.6ms inference, 49.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 Birds, 981.0ms
Speed: 28.7ms preprocess, 981.0ms inference, 42.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 Birds, 724.2ms
Speed: 17.1ms preprocess, 724.2ms inference, 14.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 Birds, 667.9ms
Speed: 16.0ms preprocess, 667.9ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 Birds, 660.8ms
Speed: 9.1ms preprocess, 660.8ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 Birds, 559.6ms
Speed: 8.6ms preprocess, 559.6ms inference, 2.6ms postprocess per ima

In [4]:
# Load YOLOv11 model
model = YOLO("my_model.pt")

# Initialize Deep SORT tracker
tracker = DeepSort(max_age=30)  # max_age=frames to keep lost object

# Load video
cap = cv2.VideoCapture('clean.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, frame.shape[1::-1])  # Resize to original size
    results = model(frame_resized)[0]

    detections = []
    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls_id))  # Deep SORT format

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame_resized)

    # Draw tracked objects
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, w, h = map(int, track.to_ltrb())
        cv2.rectangle(frame_resized, (l, t), (l + w, t + h), (0, 255, 0), 2)
        cv2.putText(frame_resized, f'Bird ID: {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Bird Detection + Tracking', frame_resized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 8 Birds, 835.7ms
Speed: 34.3ms preprocess, 835.7ms inference, 26.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Birds, 447.3ms
Speed: 11.0ms preprocess, 447.3ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 Birds, 312.7ms
Speed: 5.1ms preprocess, 312.7ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Birds, 376.8ms
Speed: 9.7ms preprocess, 376.8ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Birds, 347.5ms
Speed: 4.5ms preprocess, 347.5ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 Birds, 370.1ms
Speed: 6.9ms preprocess, 370.1ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 Birds, 344.9ms
Speed: 5.0ms preprocess, 344.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 Birds, 363.3ms
Speed: 7.5ms preprocess, 363.3ms inference, 2.4ms postprocess per image at shap